In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.io.img_tiles import GoogleTiles

from geoscripts import gchemplots as gp
from geoscripts import maps

In [ ]:
# Read in organized data
data = pd.read_csv('processed/data.csv',index_col=0)

# Filter for J and mafic
filter_period = (data['Period']=='Jurassic')|(data['Period']=='Jurassic?')|(data['Period']=='Cretaceous')|(data['Period']=='Tertiary?')
filter_major =  (data['SiO2']<100)

data_filtered = data[filter_period&filter_major]

print(data_filtered.index)
[print(len(data_filtered.index))]

In [ ]:
# Groups
khaishi_core_smps = ['G22034','G22035','G22036A','G22037','G22038','G22039']
data_filtered.loc[khaishi_core_smps,'S_Domain'] = 'Khaishi Core'

limb_condition = (~data_filtered.index.isin(khaishi_core_smps)) & (data_filtered['S_Domain'] == 'Khaishi')
data_filtered.loc[limb_condition,'S_Domain'] = 'Khaishi Limbs'

cret_condition = (data_filtered['Period'] == 'Cretaceous')|(data['Period']=='Tertiary?')
data_filtered.loc[cret_condition,'S_Domain'] = 'Cretaceous Intrusive'

group_dict = {'Khaishi Core':'Khaishi Core','Khaishi Limbs':'Khaishi Limbs','Idliani':'Idliani','Jvari':'Jvari','Lentekhi':'Lentekhi/Panaga','Mazashi':'Mazashi','Panaga':'Lentekhi/Panaga',
              'Tsageri-Khvamli':'Tsageri-Khvamli','Dzirula':np.nan,'Pasanauri':np.nan,'Dizi':np.nan,'Cretaceous Intrusive':np.nan}

chem_group = [group_dict[x] for x in data_filtered['S_Domain']]

groups = data_filtered.groupby(chem_group)

In [ ]:
# Group properties
axes_dict = {'Khaishi Core':(0,0),'Khaishi Limbs':(0,0),'Idliani':(1,0),'Jvari':(2,0),'Lentekhi/Panaga':(0,1),'Mazashi':(1,1),'Tsageri-Khvamli':(2,1)}

color_dict = {'Khaishi Core':'#88CCEE','Khaishi Limbs':'#44AA99','Idliani':'#117733','Jvari':'#999933','Lentekhi/Panaga':'#AA4499','Mazashi':'#882255','Tsageri-Khvamli':'#CC6677',
              np.nan:'#DDDDDD'}

# color_dict = {'Khaishi Core':'#762A83','Khaishi Limbs':'#9970AB','Idliani':'#C2A5CF','Jvari':'#E7D4E8','Lentekhi/Panaga':'#1B7837','Mazashi':'#5AAE61','Tsageri-Khvamli':'#ACD39E'}

In [ ]:
# Majors
fig,axs = plt.subplots(3,2,dpi=300,figsize=(6.5,7))

for k, group in groups:
    ax = axs[axes_dict[k]]
    color = color_dict[k]
    SiO2,Na2O,K2O = group[['SiO2','Na2O','K2O']].T.values
    gp.TASsm(SiO2,Na2O,K2O,label=k,ax=ax,color=color,first=[])
    ax.set_title(k)

plt.tight_layout()
fig.savefig('TAS.pdf')

In [ ]:
# Harkers

for k, group in groups:
    fig,axs = plt.subplots(4,2,figsize=(6.5,9),dpi=300)
    fig.suptitle(k)
    harker_cols = group.columns.str.replace('*','t')
    data_harker = group.copy()
    data_harker.columns = harker_cols

    gp.harker(data_harker,fig=fig,axs=axs,color=color_dict[k])
    fig.savefig(k[0:3]+'_harker.pdf')
    plt.close()

In [ ]:
# Spiders

fig0,axs0 = plt.subplots(3,2,figsize=(6.5,7),dpi=300)

fig1,axs1 = plt.subplots(3,2,figsize=(6.5,7),dpi=300)


axs_all = [axs0,axs1]

for k, group in groups:
    fignum = axes_dict[k][1]
    axs = axs_all[fignum]

    ax0 = axs[(axes_dict[k][0],0)]
    ax1 = axs[(axes_dict[k][0],1)]

    ax0.set_title(k)

    gp.REE(group,ax=ax0,c=color_dict[k])
    gp.immobile(group,ax=ax1,c=color_dict[k])

fig0.set_layout_engine('tight')
fig1.set_layout_engine('tight')

fig0.savefig('west_spiders.pdf')
fig1.savefig('east_spiders.pdf')



In [ ]:
# Ternary
fig = plt.figure(figsize=(6.5,3),dpi=300)
ax0 = fig.add_subplot(121,projection='ternary')
ax1 = fig.add_subplot(122,projection='ternary')
axs = [ax0,ax1]

for k,group in groups:
    ax = axs[axes_dict[k][1]]
    Tb,Th,Ta = group[['Tb','Th','Ta']].T.values
    gp.cabanis(Tb,Th,Ta,ax=ax,label=k,c=color_dict[k])

lgd = fig.legend()
plt.tight_layout()
fig.savefig('ternary.pdf',bbox_extra_artists=(lgd,),bbox_inches='tight')

In [ ]:
# Locations
img = GoogleTiles(style='satellite')

fig = plt.figure(figsize=(6.5,5),dpi=300)
ax0 = fig.add_subplot(121,projection=ccrs.PlateCarree())
ax1 = fig.add_subplot(122,projection=ccrs.PlateCarree())

axs = [ax0,ax1]

for ax in axs:
    ax.add_image(img,12)
    maps.shpplt_simple(shp='map_data/gc_majorfaults_wgs84.shp',edgecolor='red',
                    facecolor='None',zorder=1,ax=ax)

for k, group in groups:
    ax = axs[axes_dict[k][1]]
    ax.scatter(group['Longitude'],group['Latitude'],c=color_dict[k],label=k)
    #for index,row in group.iterrows():
    #    ax.annotate(index,(row['Longitude'],row['Latitude']))



lgd = fig.legend()
fig.savefig('locations.pdf',bbox_extra_artists=(lgd,),bbox_inches='tight')

In [ ]:
# Mantle Array Plot
# Pearce08 Fields (for future function)
from matplotlib.patches import Polygon
# MORB-OIB Array
x = [0.1,0.3,1000,1000,800,0.1]
y = [0.01,0.01,48,100,100,0.01]
xy_array = np.column_stack((x,y))

# Arc Array
b = (np.log10(10)-np.log10(1.2))/(np.log10(0.8)-np.log10(0.1))
a = np.log10(10)-np.log10(0.8)
xvals = np.arange(0,1000)
yvals = a*np.power(xvals,b)

fig,ax = plt.subplots(1,figsize=(4,4),dpi=300)

for k, group in groups:
    ax.scatter(group['Nb']/group['Yb'],group['Th']/group['Yb'],c=color_dict[k],
               label=k)
    ax.set_xscale('log')
    ax.set_yscale('log')
    pgon = Polygon(xy_array,alpha=0.2,zorder=0,color='gray')
    ax.add_patch(pgon)
    ax.plot(xvals,yvals,color='gray')
    ax.set_xlim(0.1,100)
    ax.set_ylim(0.01,10)
    ax.set_xlabel('Nb/Yb')
    ax.set_ylabel('Th/Yb')
    lgd = ax.legend()

fig.savefig('mantlearray.pdf',bbox_extra_artists=(lgd,),bbox_inches='tight')

In [ ]:
# Export locations with colors
odata = data_filtered.loc[:,['Latitude','Longitude','Latitude','Longitude','Rock_Type','Period','S_Domain']]
odata.loc[:,'Chemistry_Group'] = chem_group
odata.loc[:,'Color'] = [color_dict[x] for x in chem_group]

odata.to_csv('locations.csv')